# (2) LSTM Model - Bisaillon (2020) Dataset

Implementing the following LSTM model.
> REF https://www.kaggle.com/code/jsvishnuj/fakenews-detection-using-lstm-neural-network 
> Dataset: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset 

This trial is to see how well the model performs on Baillon's (2020) dataset. I will be using a merge of these datasets that was input in the BERT model used on Google Colab for this project (fake and true combined). As I was not able to complete the full training on the BERT model for the full set of text, so I will implement this with only the titles. This will also enable me to see how accurately the LSTM model is on a small sets of data and how long the model runs for in comparison to the full text data the LSTM model was initially trained on. 

I decided not to rerun the GRU model as it was unneccesary for further research. 

In [27]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install matplot lib

Note: you may need to restart the kernel to use updated packages.


In [29]:
# importing necessary libraries 
import pandas as pd
import tensorflow as tf
import os
import re
import numpy as np
from string import punctuation
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [30]:
# importing neural network libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU, LSTM, RNN, SpatialDropout1D

In [31]:
##importing extra libaries I may need
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [32]:
#need to do:
#import the google colab BERT true and fake datasets
#add labels to each accordingly for true and fake
#merge the datasets
#split into test and train

In [33]:
# Colab BERT true and fake datasets
bert_true = pd.read_csv(r"C:\Users\luoco\Documents\Dissertation-Project\fake-news-data\Google-Fine-Tune_BERT\a1_True.csv")
bert_fake = pd.read_csv(r"C:\Users\luoco\Documents\Dissertation-Project\fake-news-data\Google-Fine-Tune_BERT\a2_Fake.csv")

#generate target and labels
bert_true['target'] = ['True']*len(bert_true)
bert_fake['target'] = ['Fake']*len(bert_fake)

In [34]:
df_true = bert_true
df_true

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",True
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",True
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",True
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",True
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",True
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",True
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",True
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",True
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",True


In [35]:
df_fake = bert_fake
df_fake

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",Fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",Fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",Fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",Fake


In [36]:
# Concatenate the DataFrames
#https://saturncloud.io/blog/solving-the-attributeerror-dataframe-object-has-no-attribute-concat-in-pandas/#:~:text=Understanding%20the%20Error&text=understand%20the%20error.-,The%20AttributeError%3A%20'DataFrame'%20object%20has%20no%20attribute%20'concat,method%2C%20but%20a%20Pandas%20function.
bert_data = pd.concat([df_true, df_fake])
bert_data

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",True
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",True
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",True
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",True
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",True
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",Fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",Fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",Fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",Fake


In [37]:
bert_data['label'] = bert_data['target'].map({'True': 0, 'Fake': 1})
bert_data

,title,text,subject,date,target,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",True,0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",True,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",True,0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",True,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",True,0
...,...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",Fake,1
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",Fake,1
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",Fake,1
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",Fake,1


In [38]:
# shuffle the DataFrame rows
#https://www.geeksforgeeks.org/pandas-how-to-shuffle-a-dataframe-rows/
df = bert_data.sample(frac = 1)
#bert_data = df
#bert_data

In [39]:
#adding a 'length' heading applied to the [[[title]]] string.
length = []
[length.append(len(str(text))) for text in df['title']]
df['length'] = length
df.head()

,title,text,subject,date,target,label,length
7790,Trump's son-in-law held talks to set up Trump ...,"(Reuters) - Donald Trump’s son-in-law, Jared K...",politicsNews,"October 17, 2016",True,0,64
12595,LOL! HOW TO TRIGGER A LIBERAL On Halloween: “M...,It probably explains why we love this guy so m...,politics,"Oct 27, 2016",Fake,1,141
11290,"Tillerson, Lavrov agree to continue North Kore...",WASHINGTON (Reuters) - The United States and R...,worldnews,"December 27, 2017",True,0,80
6003,The Obamas Gave A Heartbreaking Tribute To Mu...,The last couple of years have been rampant wit...,News,"June 5, 2016",Fake,1,64
10734,Obama: China knows it cannot sustain export-dr...,WASHINGTON (Reuters) - U.S. President Barack O...,politicsNews,"February 22, 2016",True,0,63


In [40]:
df = df.drop(columns = ['text', 'subject', 'date'])
df

,title,target,label,length
7790,Trump's son-in-law held talks to set up Trump ...,True,0,64
12595,LOL! HOW TO TRIGGER A LIBERAL On Halloween: “M...,Fake,1,141
11290,"Tillerson, Lavrov agree to continue North Kore...",True,0,80
6003,The Obamas Gave A Heartbreaking Tribute To Mu...,Fake,1,64
10734,Obama: China knows it cannot sustain export-dr...,True,0,63
...,...,...,...,...
2959,VP Pence hires political operative as top aide...,True,0,57
4991,Republican senators wobble on House Obamacare ...,True,0,50
11421,Saudi Arabia says Shi'ite judge killed by kidn...,True,0,52
11192,TABLES TURNED! CONSERVATIVE GROUP FILES ETHICS...,Fake,1,103


In [41]:
min(df['length']), max(df['length']), round(sum(df['length'])/len(df['length']))

(8, 286, 80)

In [42]:
max_features = 4500
#keep this the same as training before on the other dataset

In [43]:
#Tokenizing the text - i.e. we are converting the words and or letters into counts or numbers. 
# We dont need to explicitly remove the punctuations. There is an inbuilt option in Tokenizer for this exact purpose
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = bert_data['title'])
X = tokenizer.texts_to_sequences(texts = bert_data['title'])

In [44]:
# now applying padding to make them even shaped.
X = pad_sequences(sequences = X, maxlen = max_features, padding = 'pre')

In [45]:
print(X.shape)
y = df['label'].values
print(y.shape)

(44898, 4500)
(44898,)


In [46]:
#https://builtin.com/data-science/train-test-split
#features = ['title']
#X = df.loc[:, features]
#y = df.loc[:, ['label']]

In [47]:
# splitting the data training data for training and validation.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 101)

In [48]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.8)

In [49]:
X

array([[   0,    0,    0, ..., 3334,  111, 2733],
       [   0,    0,    0, ...,    7, 1599,  771],
       [   0,    0,    0, ...,  898,  271,   26],
       ...,
       [   0,    0,    0, ...,    3,  950,  129],
       [   0,    0,    0, ...,   67,   39, 1543],
       [   0,    0,    0, ...,   11, 3841,  231]])

In [50]:
y

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [51]:
# LSTM Neural Network
lstm_model = Sequential(name = 'lstm_nn_model')
lstm_model.add(layer = Embedding(input_dim = max_features, output_dim = 120, name = '1st_layer'))
lstm_model.add(layer = LSTM(units = 120, dropout = 0.2, recurrent_dropout = 0.2, name = '2nd_layer'))
lstm_model.add(layer = Dropout(rate = 0.5, name = '3rd_layer'))
lstm_model.add(layer = Dense(units = 120,  activation = 'relu', name = '4th_layer'))
lstm_model.add(layer = Dropout(rate = 0.5, name = '5th_layer'))
lstm_model.add(layer = Dense(units = len(set(y)),  activation = 'sigmoid', name = 'output_layer'))
# compiling the model
lstm_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [52]:
lstm_model_fit = lstm_model.fit(X_train, y_train, epochs = 1)
#started training at 20:00 Weds 16th.

1123/1123 [==============================] - 428999s 382s/step - loss: 0.6928 - accuracy: 0.5205


#### One Epoch (cycle) took around 120 hours / 5 days 
#### loss: 0.6928
#### accuracy: 0.520
Given that the model took almost 5 days to complete running on a CPU, and the accuracy is a mere 0.5205, the model does not seem very well trained. It achieved an accuracy score lower than when the model was trained on the original dataset (0.5576). The loss is quite significant as well. Typically I would like to see a loss as little as possible/ close to 0. 
# _______________________________________________________

In [55]:
#original code
# lstm_prediction = lstm_model.predict_classes(test_text)
# issue fix ref: https://github.com/keras-team/keras/issues/15838


lstm_prediction = lstm_model.predict(X_test)

281/281 [==============================] - 317s 1s/step


In [56]:
lstm_prediction.shape

(8980, 2)